In [159]:
import csv 
import random 
import networkx as nx
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline 
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 32}



plt.rc('font', **font) 

G0 = nx.read_weighted_edgelist("/home/achiso/MATH27/4932.protein.links.v12.0.txt",comments="#",nodetype=str) #Delete the header first or this might not work!
G0 

In [680]:
threshold_score = 500
for edge in G0.edges: 
    weight = list(G0.get_edge_data(edge[0],edge[1]).values())
    if(weight[0] <= threshold_score):
        G0.remove_edge(edge[0],edge[1])

print("( O v O)b") 

( O v O)b


In [681]:
print(G0.number_of_nodes())
largest_cc = max(nx.connected_components(G0) , key=len) 
Gc = G0.subgraph(largest_cc) 
print(Gc.number_of_nodes()) 

6538
5697


In [682]:
commus = nx.community.label_propagation_communities(Gc) 
print("commu # :" , len(commus))
modularity_v1 = nx.community.modularity(Gc , commus) 

print("modularity :" , modularity_v1) 


G = nx.Graph(Gc) 
for i in commus : 
    if len(i) < 2 : 
        for ii in i : 
            G.remove_node(ii) 
print("////////// REMOVAL ///////////") 
print("\t|G0| :" , G0.number_of_nodes()) 
print("\t|Gc| :" , Gc.number_of_nodes()) 
print("\t|G| :" , G.number_of_nodes()) 

newer_commus = nx.community.label_propagation_communities(G) 
modularity_v2 = nx.community.modularity(G , newer_commus) 

print("new modularity :" , modularity_v2) 

commu # : 334
modularity : 0.5368602914379882
////////// REMOVAL ///////////
	|G0| : 6538
	|Gc| : 5697
	|G| : 5697
new modularity : 0.5368602914379882


In [683]:
def run_many_times(n=101 , G=G) : 
    run_records = [] 
    for i in range(n) : 
        random_number_in_range = random.randint(0 , 2**(64)-1) 
        rv = nx.community.fast_label_propagation_communities(G , seed=random_number_in_range) 
        rv_list = [] 
        
        
        for ii in rv : 
            rv_list.append(ii) 
        #for i in rv_list : 
        #    print(i) 
        
        run_records.append([rv_list , random_number_in_range]) 
        
        print("                    " , end='\r') 
        print("( O v O) {" , i+1 , ")" , end='\r') 

    
    return run_records 

In [684]:
def cal_rela_counters(runs , G=G) : 
    rela_c = {} 
    for i in G : 
        #rela_c.append( [i , [0]*len(G)] ) 
        rela_c[i] = [0]*len(G) 

    """   
    for i in runs : 
        for ii in G : 
            for iii in range(len(i[0])) : #result_i 
                if ii in i[0][iii] : 
                    for iiii in i[0][iii] : 
                        if ii == iiii : 
                            continue 
                        #rela_c[ii][1][iiii] += 1 
                        rela_c[ii][0] += 1 
    """
    return rela_c 

In [685]:
def top_few_btween_member(Gin , lookfor = "" , show=0) : 
    if len(Gin) < 5 : 
        #2/0 
        tmp_l = [i for i in Gin] 
        return tmp_l 
        
    tmp_btweenCentra = nx.betweenness_centrality(Gin) 
    tmp_L2 = [] 
    tmp_L1 = [] 
    pack = [] 
    for i in Gin : 
        tmp_L1.append(tmp_btweenCentra[i]) 
        tmp_L2.append(i) 

    tmp_L1, tmp_L2 = zip(*sorted(zip(tmp_L1, tmp_L2)))   ### Thanks to https://stackoverflow.com/questions/9764298/given-parallel-lists-how-can-i-sort-one-while-permuting-rearranging-the-other 

    
    for i in range(5) : 
        if show : 
            print("[ " + tmp_L2[-(1+i)] , "|" + str(tmp_L1[-(1+i)]) + "]") 
        pack.append(tmp_L2[-(1+i)]) 
    if len(lookfor) > 0 : 
        if lookfor in tmp_L2 : 
            for i in tmp_L2 : 
                if i == lookfor : 
                    print("#" + str(len(tmp_L2) - tmp_L2.index(lookfor)) , "\t" , lookfor) 
        else : 
            print("( O _ O) { ??? )") 
    return pack  

In [686]:
def tmp_F(runs , in_G=G) :  
    rela_c = {} 
    for i in in_G : 
        rela_c[i] = {} 
        for ii in in_G : 
            rela_c[i][ii] = 0 

    tmp_c = 0 
    for iiii in runs : 
        for i in in_G : 
            for ii in iiii[0] : 
                if i in ii : 
                    for iii in ii : 
                        if i == iii : 
                            continue 
                        rela_c[i][iii] += 1 
        tmp_c += 1 
        print(str(tmp_c) + "/" + str(len(runs)) + " RUN\t[ OK ]" , end='\r') 

    return rela_c 

In [687]:
def dense_commu_of(tar_node , rela_counter_in , run_result) : 
    dense_commu = [] 
    drifters = [] 
    for i in rela_counter_in[tar_node] : 
        if rela_counter_in[tar_node][i]/len(run_result) >= 0.95 :        #overlap 95% of times 
            dense_commu.append(i) 
        elif rela_counter_in[tar_node][i]/len(run_result) >= 0.5 : 
            if i != tar_node : 
                drifters.append(i) 
            
    return [dense_commu , drifters] 

In [688]:
def QC_Idea_v1(relas , runs) : 
    
    self_def_commus = [] 
    
    all_nodes = [] 
    all_nodes_info = {} 
    for i in relas : 
        all_nodes.append( i ) 
        all_nodes_info[i] = [0 , -1 , []]    # [?dense , dense # , belong] 
#    print(len(relas) , len(all_nodes) , len(all_nodes_info)) 


    new_commu_id_counter = 0 
    for i in relas : 
        cur_tar = i 
        if all_nodes_info[cur_tar][0] == 1 :      # already dense 
            continue 
        else : 
            tmp = dense_commu_of(cur_tar , relas , runs) 
            raw_dense_list = tmp[0] 
            raw_drifter_list = tmp[1] 
            new_commu_pack = [cur_tar] 
            for i in raw_dense_list : 
                if 0 == all_nodes_info[i][0] : 
                    new_commu_pack.append(i) 
            for i in new_commu_pack : 
                all_nodes_info[i][0] = 1 
                all_nodes_info[i][1] = new_commu_id_counter 
                all_nodes_info[i][2].append(new_commu_id_counter) 
            for i in raw_drifter_list : 
                all_nodes_info[i][2].append(new_commu_id_counter) 

            self_def_commus.append(new_commu_pack) 

            #print(len(raw_drifter_list) , len(raw_drifter_list) + len(new_commu_pack)) 


        new_commu_id_counter += 1 
        #print("( O A O) { STOP )") 
        #break 
    print(new_commu_id_counter) 
    return [self_def_commus , all_nodes_info] 


In [689]:
#rela_counter IN 
#run_result IN 

def QC_Idea_v2(relas , runs , starting_node="4932.YMR190C") : 
    
    self_def_commus = [] 
    
    all_nodes = [] 
    all_nodes_info = {} 
    for i in relas : 
        all_nodes.append( i ) 
        all_nodes_info[i] = [0 , -1 , []]    # [?dense , dense # , belong] 
    #print(">>" , len(all_nodes_info)) 

    new_commu_id_counter = 0 

    if len(starting_node) > 0 : 
        cur_tar = starting_node 
    

        raw_dense_list , raw_drifter_list = dense_commu_of(cur_tar , relas , runs) 

        new_commu_pack = [cur_tar] 
        for ii in raw_dense_list : 
            #if 0 == all_nodes_info[ii][0] : 
            new_commu_pack.append(ii) 
        for ii in new_commu_pack : 
            all_nodes_info[ii][0] = 1 
            all_nodes_info[ii][1] = new_commu_id_counter 
            all_nodes_info[ii][2].append(new_commu_id_counter) 
        for ii in raw_drifter_list : 
            all_nodes_info[ii][2].append(new_commu_id_counter) 
    
        self_def_commus.append(new_commu_pack) 
    
    
        new_commu_id_counter = 1 


    for i in relas : 
        cur_tar = i 
        if all_nodes_info[cur_tar][0] == 1 :      # already dense 
            continue 
#        elif len(all_nodes_info[cur_tar][2]) > 0 :    # drifter 
#            continue 
        else : 
            tmp = dense_commu_of(cur_tar , relas , runs) 
            raw_dense_list = tmp[0] 
            raw_drifter_list = tmp[1] 
            new_commu_pack = [cur_tar] 

            if len(raw_dense_list) < 11 : 
                continue 
            
            for ii in raw_dense_list : 
                if 0 == all_nodes_info[ii][0] : 
                    new_commu_pack.append(ii) 
                    
            if len(new_commu_pack) > 10 : 
                self_def_commus.append(new_commu_pack) 
            else : 
                continue 
                
            for ii in new_commu_pack : 
                all_nodes_info[ii][0] = 1 
                all_nodes_info[ii][1] = new_commu_id_counter 
                all_nodes_info[ii][2].append(new_commu_id_counter) 
            for ii in raw_drifter_list : 
                all_nodes_info[ii][2].append(new_commu_id_counter) 

#            if len(new_commu_pack) > 10 : 
#                self_def_commus.append(new_commu_pack) 
#            else : 
#                continue 

            #print(len(raw_drifter_list) , len(raw_drifter_list) + len(new_commu_pack)) 

        #print(new_commu_id_counter) 
        new_commu_id_counter += 1 

            
        
    return [self_def_commus , all_nodes_info] 


In [690]:
#def possi_dests(ni , dense_commus=tmp_box , nodes_info=tmp_pox) : 
#              starting node  ,  self_def_commus  ,  all_nodes_info 
def possi_dests(ni , dense_commus , nodes_info) : 

    trig = 0 
    
    ni_idx = 0 
    for i in dense_commus : 
        if ni in i : 
            print("[" + str(ni) + "]" , "@" , ni_idx)
            break 
        ni_idx += 1 


    possi_padding = [] 
    possi_dest = [] 
    feelted_possi_dest = [] 
    feelted_possi_padding = [] 

    
    for i in nodes_info : 
        if (ni_idx in nodes_info[i][2]) and nodes_info[i][0] == 0 :    #nodes_info[i][0] == 0 and 
            # collect drifters 
            possi_padding.append(i) 
            for ii in nodes_info[i][2] : 
                if ii not in possi_dest and ii != ni_idx : 
                    possi_dest.append(ii) 
    #print(len(possi_padding) , possi_dest) 
    #print(len(dense_commus)) 
    #print("~~~~~~~~~~~~") 
    for i in possi_dest : ########## 
        
        if len(dense_commus[i]) >= 2 : 
            feelted_possi_dest.append(i) 
    if 0 == len(feelted_possi_dest) : 
        print("( ' ^ ') { EMPTY )") 
        trig = 1 

    #print(feelted_possi_dest) 
    #print("paddings :" , str(len(possi_padding)) + "<>" + str(len(tmp_box[ni_idx])) , ": dense") 
    if trig : 
        return [possi_dest , possi_padding] 
    return [feelted_possi_dest , possi_padding] 
            

In [691]:
tar_node = "4932.YMR190C" 

In [692]:
run_result = run_many_times(101 , G) 
rela_counter = tmp_F(run_result , G) 

In [693]:
#tmp = QC_Idea_v2(rela_counter , run_result , "4932.YMR190C") 
tmp = QC_Idea_v2(rela_counter , run_result , tar_node) 
self_def_commus = tmp[0]    #self def commus 
all_node_info = tmp[1]     # info 


tar_idx = 0 
for i in self_def_commus : 
    if tar_node in i : 
        print("[" + tar_node + "]" , "@" , tar_idx) 
        break 
    tar_idx += 1 





[4932.YMR190C] @ 0


In [694]:
print(len(self_def_commus)) 
tmp_l = [] 
for i in all_node_info :
    for ii in all_node_info[i][2] : 
        tmp_l.append(ii) 
print(max(tmp_l)) 

87
86


In [695]:
feelted_self_def_commus = [] 
for i in self_def_commus : 
    if len(i) > 7 : 
        feelted_self_def_commus.append(i) 

print(" |pre feelter| =" , len(self_def_commus)) 
print("|post feelter| =" , len(feelted_self_def_commus)) 


 |pre feelter| = 87
|post feelter| = 87


In [696]:
other_sides , paddings = possi_dests(tar_node , self_def_commus , all_node_info) 

print(other_sides) 
print(len(paddings)) 

[4932.YMR190C] @ 0
[15, 27, 52, 78, 65, 55, 76, 8]
47


In [697]:
""" 
tmp_c = 0 
for i in all_node_info : 
    print(all_node_info[i]) 
    tmp_c += 1 
    if tmp_c > 101 : 
        break 
""" 

' \ntmp_c = 0 \nfor i in all_node_info : \n    print(all_node_info[i]) \n    tmp_c += 1 \n    if tmp_c > 101 : \n        break \n'

In [698]:

print() 
for i in other_sides : 
    tmp_pack = [] 
    dd_list = [] 
    tmp_G = G.subgraph(self_def_commus[i]) 
    

    for ii in paddings : 
        if i in all_node_info[ii][2] : 
            tmp_pack.append(ii) 
    tmp_top_btw = top_few_btween_member(tmp_G , "" , 0) 
    print("   starting point" , ":\t" , "4932.YMR190C") 
    #print("\tdrifters :\t" , tmp_pack) 
    print("essens 0f commu_" + str(i) + " :\t" , tmp_top_btw) 

    for ii in self_def_commus[tar_idx] : 
        dd_list.append(ii) 
    for ii in self_def_commus[i] : 
        dd_list.append(ii) 
    for ii in tmp_pack : 
        dd_list.append(ii) 
    dd_G = G.subgraph(dd_list) 

    for i in range(len(tmp_top_btw)) : 
        try : 
            print(">>>LinkingPath " + str(i+1) + " : HEAD --> TAIL\n\t\t" , nx.shortest_path(dd_G , "4932.YMR190C" , tmp_top_btw[i]) ) 
            #break 
        except nx.NetworkXNoPath : 
            print("can't reach" , "[" + tmp_top_btw[i] + "]") 
            if i == len(tmp_top_btw) - 1 : 
                print("[ NO PATH ]") 
            #print("/////////////////////////////////////////\n////////////// NO PATH /////////////////\n///////////////////////////////////") 
            
        
    #print(">>>LinkingPath : HEAD --> Drifter\n\t\t" , nx.shortest_path(G , "4932.YMR190C" , "4932.YDR545W") ) 
        
    print("\n\n") 




   starting point :	 4932.YMR190C
essens 0f commu_15 :	 ['4932.YBR010W', '4932.YBR009C', '4932.YNL031C', '4932.YBR245C', '4932.YHR119W']
>>>LinkingPath 1 : HEAD --> TAIL
		 ['4932.YMR190C', '4932.YGL058W', '4932.YBR010W']
>>>LinkingPath 2 : HEAD --> TAIL
		 ['4932.YMR190C', '4932.YJL115W', '4932.YBR009C']
>>>LinkingPath 3 : HEAD --> TAIL
		 ['4932.YMR190C', '4932.YGL058W', '4932.YNL031C']
>>>LinkingPath 4 : HEAD --> TAIL
		 ['4932.YMR190C', '4932.YAL019W', '4932.YBR245C']
>>>LinkingPath 5 : HEAD --> TAIL
		 ['4932.YMR190C', '4932.YGL058W', '4932.YHR119W']



   starting point :	 4932.YMR190C
essens 0f commu_27 :	 ['4932.YLR274W', '4932.YBL023C', '4932.YPR135W', '4932.YPR019W', '4932.YGL201C']
>>>LinkingPath 1 : HEAD --> TAIL
		 ['4932.YMR190C', '4932.YKL113C', '4932.YLR274W']
>>>LinkingPath 2 : HEAD --> TAIL
		 ['4932.YMR190C', '4932.YBL023C']
>>>LinkingPath 3 : HEAD --> TAIL
		 ['4932.YMR190C', '4932.YPR135W']
>>>LinkingPath 4 : HEAD --> TAIL
		 ['4932.YMR190C', '4932.YKL113C', '4932

In [699]:
  #                      other_sides   paddings   all_node_info   self_def_commus 
def tmp_H(starting_node , to_commus , drifters , node_infos , dense_commus , G=G): 

    idx = 0 
    for i in dense_commus : 
        if starting_node in i : 
            break 
        idx += 1 

    for i in to_commus : 
        tmp_pack = [] 
        dd_list = [] 
        tmp_G = G.subgraph(dense_commus[i]) 
        
    
        for ii in drifters : 
            if i in node_infos[ii][2] : 
                tmp_pack.append(ii) 
        tmp_top_btw = top_few_btween_member(tmp_G , "" , 0) 
        print("   starting point" , ":\t" , starting_node) 
        #print("\tdrifters :\t" , tmp_pack) 
        print("essens 0f commu_" + str(i) + " :\t" , tmp_top_btw) 
    
        for ii in dense_commus[idx] : 
            dd_list.append(ii) 
        for ii in dense_commus[i] : 
            dd_list.append(ii) 
        for ii in tmp_pack : 
            dd_list.append(ii) 
        dd_G = G.subgraph(dd_list) 
    
        for i in range(len(tmp_top_btw)) : 
            try : 
                print(">>>LinkingPath " + str(i+1) + " : HEAD --> TAIL\n\t\t" , nx.shortest_path(dd_G , starting_node , tmp_top_btw[i]) ) 
                #break 
            except nx.NetworkXNoPath : 
                print("can't reach" , "[" + tmp_top_btw[i] + "]") 
                if i == len(tmp_top_btw) - 1 : 
                    print("[ NO PATH ]") 
                #print("/////////////////////////////////////////\n////////////// NO PATH /////////////////\n///////////////////////////////////") 
                
            
        #print(">>>LinkingPath : HEAD --> Drifter\n\t\t" , nx.shortest_path(G , "4932.YMR190C" , "4932.YDR545W") ) 
            
        print("\n\n") 



In [700]:
#                      other_sides   paddings   all_node_info   self_def_commus 

next_target_node = "4932.YLR274W" 
other_sides , paddings = possi_dests(next_target_node , self_def_commus , all_node_info) 
tmp_H(next_target_node , other_sides , paddings , all_node_info, self_def_commus , G) 

[4932.YLR274W] @ 27
   starting point :	 4932.YLR274W
essens 0f commu_0 :	 ['4932.YER095W', '4932.YML032C', '4932.YHL022C', '4932.YMR190C', '4932.YFL003C']
>>>LinkingPath 1 : HEAD --> TAIL
		 ['4932.YLR274W', '4932.YKL113C', '4932.YER095W']
>>>LinkingPath 2 : HEAD --> TAIL
		 ['4932.YLR274W', '4932.YKL113C', '4932.YML032C']
>>>LinkingPath 3 : HEAD --> TAIL
		 ['4932.YLR274W', '4932.YNL088W', '4932.YHL022C']
>>>LinkingPath 4 : HEAD --> TAIL
		 ['4932.YLR274W', '4932.YKL113C', '4932.YMR190C']
>>>LinkingPath 5 : HEAD --> TAIL
		 ['4932.YLR274W', '4932.YBR088C', '4932.YER041W', '4932.YFL003C']



   starting point :	 4932.YLR274W
essens 0f commu_15 :	 ['4932.YBR010W', '4932.YBR009C', '4932.YNL031C', '4932.YBR245C', '4932.YHR119W']
>>>LinkingPath 1 : HEAD --> TAIL
		 ['4932.YLR274W', '4932.YBR010W']
>>>LinkingPath 2 : HEAD --> TAIL
		 ['4932.YLR274W', '4932.YBR009C']
>>>LinkingPath 3 : HEAD --> TAIL
		 ['4932.YLR274W', '4932.YNL031C']
>>>LinkingPath 4 : HEAD --> TAIL
		 ['4932.YLR274W', '49

In [701]:
2/0 

ZeroDivisionError: division by zero